# PageRank

*参考：《统计学习方法（第2版）》 李航*

在实际应用中许多数据都以图（graph）的形式存在，比如，互联网、社交网络都可以看作是一个图。图数据上的机器学习具有理论与应用上的重要意义。 PageRank 算法是图的链接分析（link analysis）的代表性算法，属于图数据上的无监督学习方法。

PageRank 算法最初作为互联网网页重要度的计算方法，1996年由 Page 和 Brin 提出，并用于谷歌搜索引擎的网页排序。事实上，PageRank 可以定义在任意有向图上，后来被应用到社会影响力分析、文本摘要等多个问题。

PageRank 算法的基本想法是在有向图上定义一个随机游走模型，即一阶马尔可夫链，描述随机游走者沿着有向图随机访问各个结点的行为。在一定条件下，极限情况访问每个结点的概率收敛到平稳分布，这时各个结点的平稳概率值就是其 PageRank 值，表示结点的重要度。PageRank 是递归定义的，PageRank 的计算可以通过迭代算法进行。 


## 1.pagerank定义
### 1.1 基础概念
#### 1.1.1 有向图
有向图（directed graph）记作 $G=(V,E)$，其中 $V$ 和 $E$ 分别表示结点和有向边的集合。
<div align=center>
<img src=directed_graph.png width=50%/>
</div>

#### 1.1.2 强连通图
如果一个有向图从其中任何一个结点出发可以到达其他任何一个结点，就称这个有向图是强连通图（strongly connected graph）。

#### 1.1.3 非周期性图
假设 $k$ 是一个大于 $1$ 的自然数，如果从有向图的一个结点出发返回到这个结点的每条路径的长度都是 $k$ 的倍数（最大公约数为 $k$ ），那么称这个结点为周期性结点。如果一个有向图不含有周期性结点，则称这个有向图为非周期性图 （aperiodic graph），否则为周期性图。
<div align=center>
<img src=periodic_graph.png width=50%/>
</div>
上图是一个周期性图。比如从A结点出发返回A，必须经过路径 A-B-C-A，可能的路径长度为 3、6、9... 最大公约数为 3，所以结点 A 是周期性结点。

<div align=center>
<img src=aperiodic_graph.png width=50%/>
</div>
上图是一个非周期性图。多了一个 B-B 的路径，因此路径长度可能为 3、4、7、8... 最大公约数为 1，找不到一个周期性结点。

#### 1.1.4 马尔科夫链
考虑一个随机变量的序列 $X = \{ X_0,X_1,...,X_t,...\}$ ，这里 $X_t$ 表示时刻 $t$ 的随机变量，$t=0,1,2,...$。每个随机变量 $X_t(t=0,1,2,...)$ 的取值集合相同，称为状态空间，表示为 $\mathcal S$。随机变量可以是离散的，也可以是连续的，以上随机变量的序列构成随机过程（stochastic process）。

假设在时刻 $0$ 的随机变量 $X_0$ 遵循分布概率 $P(X_0)=\pi_0$，称为初始状态分布。在某个时刻 $t \geq 1$ 的随机变量 $X_t$ 与前一个时刻的随机变量 $X_{t-1}$ 之间有条件分布 $P(X_t|X_{t-1})$，如果 $X_t$ 只依赖于 $X_{t-1}$，而不依赖于过去的随机变量 $\{X_0,X_1,...,X_{t-2}\}$，这一性质称为马尔可夫性，即
$$P(X_t|X_0,X_1,...,X_{t-1}) = P(X_t|X_{t-1}),   t=1,2,...$$
具有马尔可夫性的随机序列 $X=\{X_0,X_1,...,X_t,...\}$ 称为马尔可夫链（Markov chain），或马尔可夫过程（Markov process）。
（*以上定义是一阶马尔可夫链*）

马尔可夫性的直观解释是“未来只依赖于现在（假设现在已知），而与过去无关”。这个假设在许多应用中是合理的。
<div align=center>
<img src=markov_chain.jpg width=75%/>
</div>

#### 1.1.5 随机游走模型
给定一个含有 $n$ 个结点的有向图，在有向图上定义随机游走（random walk）模型，即一阶马尔可夫链，其中结点表示状态，有向边表示状态之间的转移，假设从一个结点到通过有向边相连的所有结点的转移概率相等。具体的，转移矩阵是一个 $n$ 阶矩阵 $M$，
$$M = [m_{ij}]_{n \times n}$$
第 $i$ 行第 $j$ 列的元素 $m_{ij}$ 取值规则如下：如果结点 $j$ 有 $k$ 个有向边连出，并且结点 $i$ 是其连出的一个结点，则 $m_{ij} = \frac{1}{k}$；否则 $m_{ij}=0, i,j=1,2,...,n$。

注意转移矩阵具有性质:
$$ m_{ij} \geq 0 $$
$$ \sum_{i=1}^{n} m_{ij} = 1 $$
即每个元素非负，每列元素之和为 $1$，即矩阵 $M$ 为随机矩阵（stochastic matrix）。

在有向图上的随机游走形成马尔可夫链。也就是说，随机游走者每经一个单位时间转移一个状态，如果当前时刻在第 $j$ 个结点（状态），那么下一个时刻在第 $i$ 个结点（状态）的概率是 $m_{ij}$，这一概率只依赖于当前的状态，与过去无关，具有马尔可夫性。
<div align=center>
<img src=directed_graph.png width=50%/>
</div>

上图可以定义随机游走模型，得到转移矩阵如下。
<div align=center>
<img src=stochastic_matrix.png width=40%/>
</div>

随机游走在某个时刻 $t$ 访问各个结点的概率分布就是马尔可夫链在时刻 $t$ 的状态分布，可以用一个 $n$ 维列向量 $R_t$ 表示，那么在时刻 $t+1$ 访问各个结点的概率分布 $R_{t+1}$ 满足
$$R_{t+1}=MR_t$$


### 1.2 PageRank的基本定义
给定一个包含 $n$ 个结点的强连通且非周期性的有向图，在有向图上定义随机游走模型，即一阶马尔可夫链。随机游走的特点是从一个结点到有有向边连出的所有结点的转移概率相等，转移矩阵为 。这个马尔可夫链具有平稳分布 


# HITS

# SALSA

# 一般PageRank的Python实现
使用第三方模块python-graph，python-graph模块实现了很多图算法，使用前需要安装，代码如下：
```
pip3 install graphviz
pip3 install python-graph-core
pip3 install python-graph-dot
```

